In [40]:
import torch
from collections import Counter
from torch import tensor


In [2]:
shuffled_nodes = [
    "47",
    "46",
    "91",
    "17",
    "68",
    "54",
    "1",
    "72",
    "22",
    "8",
    "75",
    "18",
    "59",
    "6",
    "74",
    "5",
    "52",
    "92",
    "78",
    "42",
    "36",
    "28",
    "38",
    "96",
    "81",
    "53",
    "84",
    "32",
    "88",
    "39",
    "16",
    "51",
    "62",
    "66",
    "76",
    "49",
    "98",
    "56",
    "86",
    "2",
    "41",
    "3",
    "44",
    "45",
    "25",
    "69",
    "64",
    "63",
    "15",
    "4",
    "99",
    "50",
    "10",
    "55",
    "23",
    "43",
    "61",
    "70",
    "29",
    "65",
    "30",
    "13",
    "14",
    "7",
    "83",
    "26",
    "80",
    "48",
    "90",
    "27",
    "0",
    "95",
    "60",
    "93",
    "57",
    "9",
    "31",
    "73",
    "85",
    "79",
    "67",
    "35",
    "33",
    "89",
    "97",
    "12",
    "58",
    "77",
    "19",
    "71",
    "87",
    "20",
    "24",
    "34",
    "37",
    "40",
    "94",
    "11",
    "21",
    "82",
]
shuffled_nodes = [int(item) for item in shuffled_nodes]


In [32]:
first = shuffled_nodes[0:10]
second = shuffled_nodes[10:20]
third = shuffled_nodes[20:30]
fourth = shuffled_nodes[30:40]
fifth = shuffled_nodes[40:50]


In [52]:
metrics_dictionary_first = [
    (
        1648,
        {
            "train_losses": [
                0.6785509288311005,
                0.6741270571947098,
                0.6717129647731781,
                0.6670925468206406,
            ],
            "train_loss": 0.6670925468206406,
            "train_loss_with_regularization": 0.6670925468206406,
            "train_accuracy": tensor(0.5978, device="cuda:0"),
            "epsilon": 2.3876306847678443,
        },
    ),
    (
        1665,
        {
            "train_losses": [
                0.4568750336766243,
                0.35160522162914276,
                0.33983470499515533,
                0.3339746966958046,
            ],
            "train_loss": 0.3339746966958046,
            "train_loss_with_regularization": 0.3339746966958046,
            "train_accuracy": tensor(0.8975, device="cuda:0"),
            "epsilon": 2.3876306847678443,
        },
    ),
    (
        1607,
        {
            "train_losses": [
                0.6671316921710968,
                0.6485496014356613,
                0.6457435190677643,
                0.6250966638326645,
            ],
            "train_loss": 0.6250966638326645,
            "train_loss_with_regularization": 0.6250966638326645,
            "train_accuracy": tensor(0.6770, device="cuda:0"),
            "epsilon": 2.3876306847678443,
        },
    ),
    (
        1540,
        {
            "train_losses": [
                0.5897017121315002,
                0.5341452807188034,
                0.5368303507566452,
                0.5281868800520897,
            ],
            "train_loss": 0.5281868800520897,
            "train_loss_with_regularization": 0.5281868800520897,
            "train_accuracy": tensor(0.7807, device="cuda:0"),
            "epsilon": 2.3876306847678443,
        },
    ),
    (
        1609,
        {
            "train_losses": [
                0.66694076359272,
                0.6447733491659164,
                0.6366224586963654,
                0.6577194780111313,
            ],
            "train_loss": 0.6577194780111313,
            "train_loss_with_regularization": 0.6577194780111313,
            "train_accuracy": tensor(0.6275, device="cuda:0"),
            "epsilon": 2.3876306847678443,
        },
    ),
    (
        1669,
        {
            "train_losses": [
                0.6414872407913208,
                0.6278668493032455,
                0.626196026802063,
                0.6275384277105331,
            ],
            "train_loss": 0.6275384277105331,
            "train_loss_with_regularization": 0.6275384277105331,
            "train_accuracy": tensor(0.6744, device="cuda:0"),
            "epsilon": 2.3876306847678443,
        },
    ),
    (
        1596,
        {
            "train_losses": [
                0.6876544952392578,
                0.6768694818019867,
                0.6744660586118698,
                0.6798231303691864,
            ],
            "train_loss": 0.6798231303691864,
            "train_loss_with_regularization": 0.6798231303691864,
            "train_accuracy": tensor(0.5671, device="cuda:0"),
            "epsilon": 2.3876306847678443,
        },
    ),
    (
        1559,
        {
            "train_losses": [
                0.673181414604187,
                0.6665755063295364,
                0.650856614112854,
                0.6711695790290833,
            ],
            "train_loss": 0.6711695790290833,
            "train_loss_with_regularization": 0.6711695790290833,
            "train_accuracy": tensor(0.5955, device="cuda:0"),
            "epsilon": 2.3876306847678443,
        },
    ),
    (
        1595,
        {
            "train_losses": [
                0.6435549557209015,
                0.617116242647171,
                0.5992724299430847,
                0.6049226671457291,
            ],
            "train_loss": 0.6049226671457291,
            "train_loss_with_regularization": 0.6049226671457291,
            "train_accuracy": tensor(0.6999, device="cuda:0"),
            "epsilon": 2.3876306847678443,
        },
    ),
    (
        1692,
        {
            "train_losses": [
                0.5992643982172012,
                0.5258204936981201,
                0.529743880033493,
                0.5216689854860306,
            ],
            "train_loss": 0.5216689854860306,
            "train_loss_with_regularization": 0.5216689854860306,
            "train_accuracy": tensor(0.7804, device="cuda:0"),
            "epsilon": 2.3876306847678443,
        },
    ),
]
metrics_dictionary_second = [
    (
        1573,
        {
            "train_losses": [
                0.36355239897966385,
                0.32401377707719803,
                0.338084451854229,
                0.3364860936999321,
            ],
            "train_loss": 0.3364860936999321,
            "train_loss_with_regularization": 0.3364860936999321,
            "train_accuracy": tensor(0.8964, device="cuda:0"),
            "epsilon": 2.3876306847678443,
        },
    ),
    (
        1667,
        {
            "train_losses": [
                0.5754712074995041,
                0.5669340640306473,
                0.561866283416748,
                0.5602353811264038,
            ],
            "train_loss": 0.5602353811264038,
            "train_loss_with_regularization": 0.5602353811264038,
            "train_accuracy": tensor(0.7338, device="cuda:0"),
            "epsilon": 2.3876306847678443,
        },
    ),
    (
        1639,
        {
            "train_losses": [
                0.6153732538223267,
                0.5162927657365799,
                0.5124350786209106,
                0.5222593694925308,
            ],
            "train_loss": 0.5222593694925308,
            "train_loss_with_regularization": 0.5222593694925308,
            "train_accuracy": tensor(0.7787, device="cuda:0"),
            "epsilon": 2.3876306847678443,
        },
    ),
    (
        1616,
        {
            "train_losses": [
                0.4161835387349129,
                0.34984681010246277,
                0.3285256698727608,
                0.36185961216688156,
            ],
            "train_loss": 0.36185961216688156,
            "train_loss_with_regularization": 0.36185961216688156,
            "train_accuracy": tensor(0.8849, device="cuda:0"),
            "epsilon": 2.3876306847678443,
        },
    ),
    (
        1653,
        {
            "train_losses": [
                0.6904057711362839,
                0.6804207861423492,
                0.6745842546224594,
                0.6755149364471436,
            ],
            "train_loss": 0.6755149364471436,
            "train_loss_with_regularization": 0.6755149364471436,
            "train_accuracy": tensor(0.6025, device="cuda:0"),
            "epsilon": 2.3876306847678443,
        },
    ),
    (
        1601,
        {
            "train_losses": [
                0.35964444000273943,
                0.021305561531335115,
                0.03209476638585329,
                0.026298448210582137,
            ],
            "train_loss": 0.026298448210582137,
            "train_loss_with_regularization": 0.026298448210582137,
            "train_accuracy": tensor(0.9962, device="cuda:0"),
            "epsilon": 2.3876306847678443,
        },
    ),
    (
        1647,
        {
            "train_losses": [
                0.5316429287195206,
                0.49963635951280594,
                0.5038078725337982,
                0.5030980706214905,
            ],
            "train_loss": 0.5030980706214905,
            "train_loss_with_regularization": 0.5030980706214905,
            "train_accuracy": tensor(0.7961, device="cuda:0"),
            "epsilon": 2.3876306847678443,
        },
    ),
    (
        1678,
        {
            "train_losses": [
                0.41899143159389496,
                0.37555188685655594,
                0.35233045369386673,
                0.3660569489002228,
            ],
            "train_loss": 0.3660569489002228,
            "train_loss_with_regularization": 0.3660569489002228,
            "train_accuracy": tensor(0.8805, device="cuda:0"),
            "epsilon": 2.3876306847678443,
        },
    ),
    (
        1668,
        {
            "train_losses": [
                0.6927303224802017,
                0.684689849615097,
                0.6799579113721848,
                0.6787733882665634,
            ],
            "train_loss": 0.6787733882665634,
            "train_loss_with_regularization": 0.6787733882665634,
            "train_accuracy": tensor(0.6062, device="cuda:0"),
            "epsilon": 2.3876306847678443,
        },
    ),
    (
        1520,
        {
            "train_losses": [
                0.617252508799235,
                0.4387496014436086,
                0.42325649162133533,
                0.4551592394709587,
            ],
            "train_loss": 0.4551592394709587,
            "train_loss_with_regularization": 0.4551592394709587,
            "train_accuracy": tensor(0.8311, device="cuda:0"),
            "epsilon": 2.7820470020134143,
        },
    ),
]
metrics_dictionary_third = [
    (
        1644,
        {
            "train_losses": [
                0.5178250446915627,
                0.4828706458210945,
                0.5040339231491089,
                0.48866449296474457,
            ],
            "train_loss": 0.48866449296474457,
            "train_loss_with_regularization": 0.48866449296474457,
            "train_accuracy": tensor(0.8015, device="cuda:0"),
            "epsilon": 2.3876306847678443,
        },
    ),
    (
        1608,
        {
            "train_losses": [
                0.6356316953897476,
                0.6087294518947601,
                0.5971973836421967,
                0.5868132412433624,
            ],
            "train_loss": 0.5868132412433624,
            "train_loss_with_regularization": 0.5868132412433624,
            "train_accuracy": tensor(0.7120, device="cuda:0"),
            "epsilon": 2.3876306847678443,
        },
    ),
    (
        1734,
        {
            "train_losses": [
                0.31607137620449066,
                0.199892096221447,
                0.2130134105682373,
                0.1920875757932663,
            ],
            "train_loss": 0.1920875757932663,
            "train_loss_with_regularization": 0.1920875757932663,
            "train_accuracy": tensor(0.9548, device="cuda:0"),
            "epsilon": 2.3876306847678443,
        },
    ),
    (
        1626,
        {
            "train_losses": [
                0.620660275220871,
                0.5827067643404007,
                0.5810171514749527,
                0.5902788788080215,
            ],
            "train_loss": 0.5902788788080215,
            "train_loss_with_regularization": 0.5902788788080215,
            "train_accuracy": tensor(0.6904, device="cuda:0"),
            "epsilon": 2.3876306847678443,
        },
    ),
    (
        1698,
        {
            "train_losses": [
                0.6075837314128876,
                0.5883896350860596,
                0.5835865288972855,
                0.5774322897195816,
            ],
            "train_loss": 0.5774322897195816,
            "train_loss_with_regularization": 0.5774322897195816,
            "train_accuracy": tensor(0.7088, device="cuda:0"),
            "epsilon": 2.3876306847678443,
        },
    ),
    (
        1738,
        {
            "train_losses": [
                0.6422561705112457,
                0.6107917428016663,
                0.598798006772995,
                0.6119734644889832,
            ],
            "train_loss": 0.6119734644889832,
            "train_loss_with_regularization": 0.6119734644889832,
            "train_accuracy": tensor(0.6777, device="cuda:0"),
            "epsilon": 2.3876306847678443,
        },
    ),
    (
        1669,
        {
            "train_losses": [
                0.669557973742485,
                0.669919490814209,
                0.6831022650003433,
                0.6486614644527435,
            ],
            "train_loss": 0.6486614644527435,
            "train_loss_with_regularization": 0.6486614644527435,
            "train_accuracy": tensor(0.5984, device="cuda:0"),
            "epsilon": 2.3876306847678443,
        },
    ),
    (
        1529,
        {
            "train_losses": [
                0.3474445197731256,
                0.03484130837023258,
                0.02637323746457696,
                0.047733698040246964,
            ],
            "train_loss": 0.047733698040246964,
            "train_loss_with_regularization": 0.047733698040246964,
            "train_accuracy": tensor(0.9927, device="cuda:0"),
            "epsilon": 2.7820470020134143,
        },
    ),
    (
        1625,
        {
            "train_losses": [
                0.34191015362739563,
                0.183582354336977,
                0.19618604332208633,
                0.17556268721818924,
            ],
            "train_loss": 0.17556268721818924,
            "train_loss_with_regularization": 0.17556268721818924,
            "train_accuracy": tensor(0.9587, device="cuda:0"),
            "epsilon": 2.3876306847678443,
        },
    ),
    (
        1595,
        {
            "train_losses": [
                0.6601380556821823,
                0.6114329546689987,
                0.6035487353801727,
                0.6102247387170792,
            ],
            "train_loss": 0.6102247387170792,
            "train_loss_with_regularization": 0.6102247387170792,
            "train_accuracy": tensor(0.6898, device="cuda:0"),
            "epsilon": 2.3876306847678443,
        },
    ),
]
metrics_dictionary_fourth = [
    (
        1577,
        {
            "train_losses": [
                0.6411599814891815,
                0.643893226981163,
                0.6313192546367645,
                0.6655671149492264,
            ],
            "train_loss": 0.6655671149492264,
            "train_loss_with_regularization": 0.6655671149492264,
            "train_accuracy": tensor(0.6404, device="cuda:0"),
            "epsilon": 2.3876306847678443,
        },
    ),
    (
        1592,
        {
            "train_losses": [
                0.6356759518384933,
                0.6336673945188522,
                0.6862501055002213,
                0.6544412821531296,
            ],
            "train_loss": 0.6544412821531296,
            "train_loss_with_regularization": 0.6544412821531296,
            "train_accuracy": tensor(0.6191, device="cuda:0"),
            "epsilon": 2.3876306847678443,
        },
    ),
    (
        1604,
        {
            "train_losses": [
                0.493204228579998,
                0.4682733863592148,
                0.4709666594862938,
                0.4793985113501549,
            ],
            "train_loss": 0.4793985113501549,
            "train_loss_with_regularization": 0.4793985113501549,
            "train_accuracy": tensor(0.8074, device="cuda:0"),
            "epsilon": 2.3876306847678443,
        },
    ),
    (
        1600,
        {
            "train_losses": [
                0.2105345670133829,
                0.11590266041457653,
                0.11851191893219948,
                0.12363487668335438,
            ],
            "train_loss": 0.12363487668335438,
            "train_loss_with_regularization": 0.12363487668335438,
            "train_accuracy": tensor(0.9749, device="cuda:0"),
            "epsilon": 2.3876306847678443,
        },
    ),
    (
        1642,
        {
            "train_losses": [
                0.5244151726365089,
                0.522714376449585,
                0.49161478132009506,
                0.5062936842441559,
            ],
            "train_loss": 0.5062936842441559,
            "train_loss_with_regularization": 0.5062936842441559,
            "train_accuracy": tensor(0.7856, device="cuda:0"),
            "epsilon": 2.3876306847678443,
        },
    ),
    (
        1646,
        {
            "train_losses": [
                0.6612961292266846,
                0.6342029571533203,
                0.614497497677803,
                0.650105744600296,
            ],
            "train_loss": 0.650105744600296,
            "train_loss_with_regularization": 0.650105744600296,
            "train_accuracy": tensor(0.6489, device="cuda:0"),
            "epsilon": 2.3876306847678443,
        },
    ),
    (
        1568,
        {
            "train_losses": [
                0.6018076092004776,
                0.5268699079751968,
                0.5320884138345718,
                0.5169631019234657,
            ],
            "train_loss": 0.5169631019234657,
            "train_loss_with_regularization": 0.5169631019234657,
            "train_accuracy": tensor(0.7770, device="cuda:0"),
            "epsilon": 2.3876306847678443,
        },
    ),
    (
        1584,
        {
            "train_losses": [
                0.5512007474899292,
                0.5424575656652451,
                0.5429843962192535,
                0.5301915109157562,
            ],
            "train_loss": 0.5301915109157562,
            "train_loss_with_regularization": 0.5301915109157562,
            "train_accuracy": tensor(0.7602, device="cuda:0"),
            "epsilon": 2.3876306847678443,
        },
    ),
    (
        1598,
        {
            "train_losses": [
                0.6319654881954193,
                0.646251454949379,
                0.6419526785612106,
                0.6321740597486496,
            ],
            "train_loss": 0.6321740597486496,
            "train_loss_with_regularization": 0.6321740597486496,
            "train_accuracy": tensor(0.6470, device="cuda:0"),
            "epsilon": 2.3876306847678443,
        },
    ),
    (
        1617,
        {
            "train_losses": [
                0.6784789264202118,
                0.6788015961647034,
                0.6822852194309235,
                0.6744879633188248,
            ],
            "train_loss": 0.6744879633188248,
            "train_loss_with_regularization": 0.6744879633188248,
            "train_accuracy": tensor(0.5719, device="cuda:0"),
            "epsilon": 2.3876306847678443,
        },
    ),
]
metrics_dictionary_fifth = [
    (
        1535,
        {
            "train_losses": [
                0.1131019901484251,
                0.03808064479380846,
                0.04172843564301729,
                0.0432131914421916,
            ],
            "train_loss": 0.0432131914421916,
            "train_loss_with_regularization": 0.0432131914421916,
            "train_accuracy": tensor(0.9928, device="cuda:0"),
            "epsilon": 2.7820470020134143,
        },
    ),
    (
        1591,
        {
            "train_losses": [
                0.6549172103404999,
                0.5692544132471085,
                0.5677401125431061,
                0.5641676932573318,
            ],
            "train_loss": 0.5641676932573318,
            "train_loss_with_regularization": 0.5641676932573318,
            "train_accuracy": tensor(0.7235, device="cuda:0"),
            "epsilon": 2.3876306847678443,
        },
    ),
    (
        1565,
        {
            "train_losses": [
                0.38274646550416946,
                0.3629138469696045,
                0.3236176520586014,
                0.34718621522188187,
            ],
            "train_loss": 0.34718621522188187,
            "train_loss_with_regularization": 0.34718621522188187,
            "train_accuracy": tensor(0.8842, device="cuda:0"),
            "epsilon": 2.3876306847678443,
        },
    ),
    (
        1575,
        {
            "train_losses": [
                0.2063360959291458,
                0.13259408995509148,
                0.1884375847876072,
                0.1421721037477255,
            ],
            "train_loss": 0.1421721037477255,
            "train_loss_with_regularization": 0.1421721037477255,
            "train_accuracy": tensor(0.9705, device="cuda:0"),
            "epsilon": 2.3876306847678443,
        },
    ),
    (
        1599,
        {
            "train_losses": [
                0.5395646318793297,
                0.39626042544841766,
                0.4033621475100517,
                0.41409899294376373,
            ],
            "train_loss": 0.41409899294376373,
            "train_loss_with_regularization": 0.41409899294376373,
            "train_accuracy": tensor(0.8455, device="cuda:0"),
            "epsilon": 2.3876306847678443,
        },
    ),
    (
        1690,
        {
            "train_losses": [
                0.6649802178144455,
                0.65748630464077,
                0.6672435253858566,
                0.6775188148021698,
            ],
            "train_loss": 0.6775188148021698,
            "train_loss_with_regularization": 0.6775188148021698,
            "train_accuracy": tensor(0.5665, device="cuda:0"),
            "epsilon": 2.3876306847678443,
        },
    ),
    (
        1692,
        {
            "train_losses": [
                0.6158180981874466,
                0.5112099200487137,
                0.49441395699977875,
                0.5020797029137611,
            ],
            "train_loss": 0.5020797029137611,
            "train_loss_with_regularization": 0.5020797029137611,
            "train_accuracy": tensor(0.7845, device="cuda:0"),
            "epsilon": 2.3876306847678443,
        },
    ),
    (
        1643,
        {
            "train_losses": [
                0.5549505054950714,
                0.44475968182086945,
                0.4427568167448044,
                0.4167347177863121,
            ],
            "train_loss": 0.4167347177863121,
            "train_loss_with_regularization": 0.4167347177863121,
            "train_accuracy": tensor(0.8409, device="cuda:0"),
            "epsilon": 2.3876306847678443,
        },
    ),
    (
        1646,
        {
            "train_losses": [
                0.3554138094186783,
                0.3281247094273567,
                0.3378807380795479,
                0.32692110538482666,
            ],
            "train_loss": 0.32692110538482666,
            "train_loss_with_regularization": 0.32692110538482666,
            "train_accuracy": tensor(0.8981, device="cuda:0"),
            "epsilon": 2.3876306847678443,
        },
    ),
    (
        1663,
        {
            "train_losses": [
                0.6719784289598465,
                0.6544035375118256,
                0.6506752222776413,
                0.6456378102302551,
            ],
            "train_loss": 0.6456378102302551,
            "train_loss_with_regularization": 0.6456378102302551,
            "train_accuracy": tensor(0.6201, device="cuda:0"),
            "epsilon": 2.3876306847678443,
        },
    ),
]


In [58]:
def compute_weighted_average(accuracies, samples):
    accuracies_weighted = [a * s for a, s in zip(accuracies, samples)]
    return sum(accuracies_weighted) / sum(samples)


In [59]:
samples = []
accuracies = []

first_datasets = []
for node_name in first:
    _, sensitive_attributes, labels = torch.load(
        f"./data/celeba/celeba-10-batches-py/federated/{node_name}/train.pt"
    )
    for label, sensitive_attribute in zip(labels, sensitive_attributes):
        first_datasets.append((label, sensitive_attribute.item()))
    accuracy = None
    for item in metrics_dictionary_first:
        if item[0] == len(labels):
            accuracy = item[1]["train_accuracy"]
    print(
        f"Total samples {len(labels)}",
        Counter(zip(labels, [item.item() for item in sensitive_attributes])),
        accuracy,
    )
    samples.append(len(labels))
    accuracies.append(accuracy)
print(f"Mean Accuracy {compute_weighted_average(accuracies, samples)}")
print(
    "____________________________________________________________________________________________"
)

samples = []
accuracies = []
second_datasets = []
for node_name in second:
    _, sensitive_attributes, labels = torch.load(
        f"./data/celeba/celeba-10-batches-py/federated/{node_name}/train.pt"
    )
    for label, sensitive_attribute in zip(labels, sensitive_attributes):
        second_datasets.append((label, sensitive_attribute.item()))
    accuracy = None
    for item in metrics_dictionary_second:
        if item[0] == len(labels):
            accuracy = item[1]["train_accuracy"]
    print(
        f"Total samples {len(labels)}",
        Counter(zip(labels, [item.item() for item in sensitive_attributes])),
        accuracy,
    )
    samples.append(len(labels))
    accuracies.append(accuracy)
print(f"Mean Accuracy {compute_weighted_average(accuracies, samples)}")

print(
    "____________________________________________________________________________________________"
)

accuracies = []
samples = []
third_datasets = []
for node_name in third:
    _, sensitive_attributes, labels = torch.load(
        f"./data/celeba/celeba-10-batches-py/federated/{node_name}/train.pt"
    )
    for label, sensitive_attribute in zip(labels, sensitive_attributes):
        third_datasets.append((label, sensitive_attribute.item()))
    accuracy = None
    for item in metrics_dictionary_third:
        if item[0] == len(labels):
            accuracy = item[1]["train_accuracy"]
    print(
        f"Total samples {len(labels)}",
        Counter(zip(labels, [item.item() for item in sensitive_attributes])),
        accuracy,
    )
    samples.append(len(labels))
    accuracies.append(accuracy)

print(f"Mean Accuracy {compute_weighted_average(accuracies, samples)}")

print(
    "____________________________________________________________________________________________"
)

accuracies = []
samples = []
fourth_datasets = []
for node_name in fourth:
    _, sensitive_attributes, labels = torch.load(
        f"./data/celeba/celeba-10-batches-py/federated/{node_name}/train.pt"
    )
    for label, sensitive_attribute in zip(labels, sensitive_attributes):
        fourth_datasets.append((label, sensitive_attribute.item()))
    accuracy = None
    for item in metrics_dictionary_fourth:
        if item[0] == len(labels):
            accuracy = item[1]["train_accuracy"]
    print(
        f"Total samples {len(labels)}",
        Counter(zip(labels, [item.item() for item in sensitive_attributes])),
        accuracy,
    )
    samples.append(len(labels))
    accuracies.append(accuracy)

print(f"Mean Accuracy {compute_weighted_average(accuracies, samples)}")

print(
    "____________________________________________________________________________________________"
)
fifth_datasets = []
samples = []
accuracies = []

for node_name in fifth:
    _, sensitive_attributes, labels = torch.load(
        f"./data/celeba/celeba-10-batches-py/federated/{node_name}/train.pt"
    )
    for label, sensitive_attribute in zip(labels, sensitive_attributes):
        fifth_datasets.append((label, sensitive_attribute.item()))
    accuracy = None
    for item in metrics_dictionary_fifth:
        if item[0] == len(labels):
            accuracy = item[1]["train_accuracy"]
    print(
        f"Total samples {len(labels)}",
        Counter(zip(labels, [item.item() for item in sensitive_attributes])),
        accuracy,
    )
    samples.append(len(labels))
    accuracies.append(accuracy)

print(f"Mean Accuracy {compute_weighted_average(accuracies, samples)}")


Total samples 1595 Counter({(0, 0): 564, (0, 1): 548, (1, 0): 311, (1, 1): 172}) tensor(0.6999, device='cuda:0')
Total samples 1607 Counter({(0, 0): 552, (0, 1): 502, (1, 0): 387, (1, 1): 166}) tensor(0.6770, device='cuda:0')
Total samples 1596 Counter({(1, 0): 468, (0, 0): 455, (0, 1): 449, (1, 1): 224}) tensor(0.5671, device='cuda:0')
Total samples 1669 Counter({(0, 1): 559, (0, 0): 554, (1, 0): 347, (1, 1): 209}) tensor(0.6744, device='cuda:0')
Total samples 1648 Counter({(1, 0): 629, (0, 0): 356, (1, 1): 351, (0, 1): 312}) tensor(0.5978, device='cuda:0')
Total samples 1692 Counter({(0, 0): 659, (0, 1): 658, (1, 0): 250, (1, 1): 125}) tensor(0.7804, device='cuda:0')
Total samples 1559 Counter({(0, 0): 500, (0, 1): 465, (1, 0): 392, (1, 1): 202}) tensor(0.5955, device='cuda:0')
Total samples 1665 Counter({(0, 1): 748, (0, 0): 747, (1, 0): 120, (1, 1): 50}) tensor(0.8975, device='cuda:0')
Total samples 1540 Counter({(1, 0): 783, (1, 1): 409, (0, 0): 183, (0, 1): 165}) tensor(0.7807, d

In [57]:
print("First: ", Counter(first_datasets))
print("Second: ", Counter(second_datasets))
print("Third: ", Counter(third_datasets))
print("Fourth: ", Counter(fourth_datasets))
print("Fifth: ", Counter(fifth_datasets))


First:  Counter({(0, 0): 4895, (0, 1): 4660, (1, 0): 4359, (1, 1): 2266})
Second:  Counter({(0, 0): 4819, (1, 0): 4727, (0, 1): 4283, (1, 1): 2433})
Third:  Counter({(1, 0): 5654, (0, 0): 4100, (0, 1): 3780, (1, 1): 2932})
Fourth:  Counter({(1, 0): 6729, (1, 1): 3444, (0, 0): 3059, (0, 1): 2796})
Fifth:  Counter({(1, 0): 5984, (0, 0): 3564, (0, 1): 3536, (1, 1): 3115})
